# Solver configuration and error control
Author: Jørgen S. Dokken

In this section, we will go through how to specify what linear algebra solver we would like to use to solve our PDEs, as well as how to verify the implemenation by considering convergence rates.

```{math}
-\Delta u &= f &&\text{in } \Omega\\
u&= u_D &&\text{on } \partial \Omega.
```
Using the manufactured solution $u_D=\cos(2\pi x)\cos(2\pi y)$, we obtain $f=8\pi^2\cos(2\pi x)\cos(2\pi y)$.
We start by creating a generic module for evaluating the analytical solution  at any point $x$.

In [1]:
import dolfinx
import numpy
import ufl

def u_ex(mod):
    return lambda x: mod.cos(2*mod.pi*x[0])*mod.cos(2*mod.pi*x[1])

Note that the return type of `u_ex` is a `lambda` function. Thus, we can create two different lambda functions, one using `numpy` (which will be used for interpolation) and one using `ufl` (which will be used for defining the source term)

In [2]:
u_numpy = u_ex(numpy)
u_ufl = u_ex(ufl)

We start by using ufl to define our source term, using `ufl.SpatialCoordinate` as input to `u_ufl`.

In [3]:
from mpi4py import MPI
mesh = dolfinx.UnitSquareMesh(MPI.COMM_WORLD, 30, 30)
x = ufl.SpatialCoordinate(mesh)
f = -ufl.div(ufl.grad(u_ufl(x)))

Next, we define our linear variational problem

In [10]:
from petsc4py import PETSc
V = dolfinx.FunctionSpace(mesh, ("CG", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx
u_bc = dolfinx.Function(V)
u_bc.interpolate(u_numpy)
u_bc.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim -1, lambda x: numpy.full(x.shape[1], True))
dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim-1, facets)
bcs = [dolfinx.DirichletBC(u_bc, dofs)]

We start by solving the problem with an LU factorization, a direct solver method (similar to Gaussian elimination).

In [9]:
default_problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs,
petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = default_problem.solve()

We now look at the solver process by inspecting the `PETSc`-solver. As the view-options in PETSc are not adjusted for notebooks (`solver.view()` will print output to the terminal if used in a `.py` file), we write the solver output to file and read it in and print the output.

In [6]:
lu_solver = default_problem.solver
viewer = PETSc.Viewer().createASCII("lu_output.txt")
lu_solver.view(viewer)
solver_output = open("lu_output.txt", "r")
for line in solver_output.readlines():
    print(line)

KSP Object: (dolfinx_solve_140206046543936) 1 MPI processes

  type: preonly

  maximum iterations=10000, initial guess is zero

  tolerances:  relative=1e-05, absolute=1e-50, divergence=10000.

  left preconditioning

  using NONE norm type for convergence test

PC Object: (dolfinx_solve_140206046543936) 1 MPI processes

  type: lu

    out-of-place factorization

    tolerance for zero pivot 2.22045e-14

    matrix ordering: nd

    factor fill ratio given 5., needed 5.0719

      Factored matrix follows:

        Mat Object: 1 MPI processes

          type: seqaij

          rows=961, cols=961

          package used to perform factorization: petsc

          total: nonzeros=32871, allocated nonzeros=32871

            not using I-node routines

  linear system matrix = precond matrix:

  Mat Object: 1 MPI processes

    type: seqaij

    rows=961, cols=961

    total: nonzeros=6481, allocated nonzeros=6481

    total number of mallocs used during MatSetValues calls=0

      not usi

This is a very robust and simple method, and is the recommended method up to a few thousand unknowns and can be efficiently used for many 2D and smaller 3D problems. However, sparse LU decomposition quickly becomes slow, as for a $N\times N$-matrix the number of floating point operations scales as $\sim (2/3)N^3$.

For large problems, we instead need to use an iterative method which are faster and require less memory. 
## Choosing a linear solver and preconditioner
As the Poisson equation results in a symmetric, positive definite system matrix, the optimal Krylov solver is the conjugate gradient (CG) method. The default preconditioner is the incomplete LU factorization (ILU), which is a popular and robous overall preconditioner. We can change the preconditioner by setting `"pc_type"` to some of the other preconditioners in petsc, which you can find in the [PETSc documentation](https://www.mcs.anl.gov/petsc/documentation/linearsolvertable.html). 
You can set any opition in `PETSc` through the `petsc_options` input, such as the absolute tolerance (`"ksp_atol"`), relative tolerance (`"ksp_rtol"`) and maximum number of iterations (`"ksp_max_it"`).

In [7]:
cg_problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs,
petsc_options={"ksp_type": "cg", "ksp_rtol":1e-6, "ksp_atol":1e-10, "ksp_max_it": 1000})
uh = cg_problem.solve()
cg_solver = cg_problem.solver
viewer = PETSc.Viewer().createASCII("cg_output.txt")
cg_solver.view(viewer)
solver_output = open("cg_output.txt", "r")
for line in solver_output.readlines():
    print(line)

KSP Object: (dolfinx_solve_140206046743040) 1 MPI processes

  type: cg

  maximum iterations=1000, initial guess is zero

  tolerances:  relative=1e-06, absolute=1e-10, divergence=10000.

  left preconditioning

  using PRECONDITIONED norm type for convergence test

PC Object: (dolfinx_solve_140206046743040) 1 MPI processes

  type: ilu

    out-of-place factorization

    0 levels of fill

    tolerance for zero pivot 2.22045e-14

    matrix ordering: natural

    factor fill ratio given 1., needed 1.

      Factored matrix follows:

        Mat Object: 1 MPI processes

          type: seqaij

          rows=961, cols=961

          package used to perform factorization: petsc

          total: nonzeros=6481, allocated nonzeros=6481

            not using I-node routines

  linear system matrix = precond matrix:

  Mat Object: 1 MPI processes

    type: seqaij

    rows=961, cols=961

    total: nonzeros=6481, allocated nonzeros=6481

    total number of mallocs used during MatSetVal

For non-symmetrix problems, a Krylov solver for non-symmetrix systems, such as GMRES is a better.

In [8]:
gmres_problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs,
petsc_options={"ksp_type": "gmres", "ksp_rtol":1e-6, "ksp_atol":1e-10, "ksp_max_it": 1000, "pc_type": "none"})
uh = gmres_problem.solve()
gmres_solver = gmres_problem.solver
viewer = PETSc.Viewer().createASCII("gmres_output.txt")
gmres_solver.view(viewer)
solver_output = open("gmres_output.txt", "r")
for line in solver_output.readlines():
    print(line)

KSP Object: (dolfinx_solve_140206046741600) 1 MPI processes

  type: gmres

    restart=30, using Classical (unmodified) Gram-Schmidt Orthogonalization with no iterative refinement

    happy breakdown tolerance 1e-30

  maximum iterations=1000, initial guess is zero

  tolerances:  relative=1e-06, absolute=1e-10, divergence=10000.

  left preconditioning

  using PRECONDITIONED norm type for convergence test

PC Object: (dolfinx_solve_140206046741600) 1 MPI processes

  type: none

  linear system matrix = precond matrix:

  Mat Object: 1 MPI processes

    type: seqaij

    rows=961, cols=961

    total: nonzeros=6481, allocated nonzeros=6481

    total number of mallocs used during MatSetValues calls=0

      not using I-node routines



```{admonition} A remark regarding verification using iterative solvers
When we consider manufactured solutions where we expect the resulting error to be of machine precision, it gets complicated when we use iterative methods. The problem is to keep the error due to the iterative solution smaller than the tolerance used in the iterative test. For linear elements and small meshes, a tolerance of between $10^{-11}$ and $10^{-12}$ works well in the case of Krylov solvers too.
```